In [1]:
# First XGBoost model
from numpy import loadtxt
from xgboost import XGBClassifier
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import numpy as np
import pandas as pd
import pickle as pk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from datetime import datetime, date
import calendar
from sklearn import ensemble
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings('ignore')
print(xgboost.__version__)

c:\Users\Russell\Anaconda3\envs\CondaApp\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


1.5.0


In [2]:
#Load Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Loading the dataset 
currentdf  = pd.read_csv(r"C:\Users\Russell\Desktop\SmartParkingApp\PL80_Dataset_1.csv")
currentdf = currentdf.dropna()
currentdf = currentdf.sample(frac=1)

print(currentdf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11548 entries, 3914 to 2689
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SESSIONID         11548 non-null  object 
 1   CARPARKNUMBER     11548 non-null  object 
 2   VEHICLETYPE       11548 non-null  object 
 3   SESSIONSTART      11548 non-null  object 
 4   SESSIONEND        11548 non-null  object 
 5   TOTAL_CHARGE      11548 non-null  float64
 6   DURATION          11548 non-null  int64  
 7   EFFECTIVE_CHARGE  11548 non-null  float64
 8   LOT_NUMBER        11548 non-null  int64  
 9   LABEL_TYPE        11548 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 992.4+ KB
None


In [3]:
#y is the output and X is the input to train
y = currentdf['LABEL_TYPE']
X = currentdf.drop(['SESSIONID','CARPARKNUMBER','LABEL_TYPE', 'LOT_NUMBER'], axis = 1)
X.head()

,VEHICLETYPE,SESSIONSTART,SESSIONEND,TOTAL_CHARGE,DURATION,EFFECTIVE_CHARGE
3914,C,23/10/2021 6:00,23/10/2021 6:01,0.00,1,0.00
9671,C,10/10/2021 4:37,10/10/2021 17:06,14.86,749,0.00
6032,C,4/8/2021 19:57,4/8/2021 21:24,1.74,87,1.74
7741,C,6/9/2021 19:21,6/9/2021 22:00,3.18,159,3.18
8910,C,29/9/2021 19:30,29/9/2021 21:24,2.28,114,2.28


In [4]:
#Used LabelEncoder to get numeric values
le = LabelEncoder()
X['VEHICLETYPE'] = le.fit_transform(X['VEHICLETYPE'])

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART']).map(pd.Timestamp.timestamp)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND']).map(pd.Timestamp.timestamp)

X['SESSIONSTART'] = X['SESSIONSTART'] / 60
X['SESSIONEND'] = X['SESSIONEND'] / 60

print(X.head(10))
print(y.head(10))

#Standardized
sc = StandardScaler()
X = sc.fit_transform(X)

#Return back to dataFrame
X = pd.DataFrame(X)

      VEHICLETYPE  SESSIONSTART  SESSIONEND  TOTAL_CHARGE  DURATION  \
3914            0    27249480.0  27249481.0          0.00         1   
9671            0    27230677.0  27231426.0         14.86       749   
6032            0    27135117.0  27135204.0          1.74        87   
7741            0    27182601.0  27182760.0          3.18       159   
8910            0    27215730.0  27215844.0          2.28       114   
1060            0    27230258.0  27231613.0          0.00      1355   
5177            0    27257717.0  27258195.0          0.00       478   
8963            0    27217217.0  27217310.0          1.86        93   
6880            0    27158423.0  27158887.0          6.34       464   
8730            0    27210009.0  27210231.0          4.44       222   

      EFFECTIVE_CHARGE  
3914              0.00  
9671              0.00  
6032              1.74  
7741              3.18  
8910              2.28  
1060              0.00  
5177              0.00  
8963              

In [5]:
#Tuning with Hyper Parameters

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

xgb_classifier = XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1,
                               colsample_bynode=1,
                               colsample_bytree=0.5,
                               enable_categorical=False,
                               eval_metric='mlogloss',
                               gamma=0.1,  
                               gpu_id=-1,
                               importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.2,
                               max_delta_step=0,
                               max_depth=15,
                               min_child_weight=1,
                               monotone_constraints='()',
                               n_estimators=100,
                               n_jobs=12, num_parallel_tree=1,
                               predictor='auto',
                               random_state=0,
                               reg_alpha=0,
                               reg_lambda=1,
                               scale_pos_weight=1,
                               subsample=1,
                               tree_method='exact',
                               validate_parameters=1,
                               verbosity=None)

xgb_classifier.fit(X_train, y_train)
y_pred_test_tuned = xgb_classifier.predict(X_test)   
accuracy_score(y_test, y_pred_test_tuned)

0.9367965367965368

In [27]:
import pickle
# create an iterator object with write permission - model.pkl
with open('XGBoost_pkl_Latest2.pkl', 'wb') as files:
    pickle.dump(xgb_classifier, files)
    
# model = pickle.load(open('XGBoost_pkl_Latest2.pkl', 'rb'))
# model.score(X_test, y_test)


xgb_classifier.save_model('model.txt')


model = xgboost.XGBClassifier()
model.load_model('model.txt')

model.score(X_test, y_pred_test_tuned)
 
model.predict(X_test)

array([1, 0, 1, ..., 0, 0, 1])